In [3]:
#| hide
from nbdev.showdoc import *

# Snowflake Feature Store
> A Pythonic interface for feature engineering in Snowflake, focusing on simplicity and type safety.



## What is this?
This library provides a clean interface to Snowflake's Feature Store, making it easy to:
1. Create and manage feature transformations
2. Handle temporal and windowed aggregations
3. Generate training datasets with point-in-time correctness
4. Maintain feature documentation and versioning



## Install

```bash
pip install snowflake-feature-store
```



## How to use



### Basic Example

Here's a minimal example of creating a feature view:

```python
from snowflake_feature_store.connection import get_connection
from snowflake_feature_store.manager import feature_store_session
from snowflake_feature_store.feature_view import FeatureViewConfig

# Connect and create feature store session
conn = get_connection()
with feature_store_session(conn) as fs:
    # Define customer features
    config = FeatureViewConfig(
        name="customer_behavior",
        domain="RETAIL",
        entity="CUSTOMER",
        feature_descriptions={"session_length": "Minutes per session"}
    )
    
    # Create customer entity
    fs.add_entity(
        name="CUSTOMER",
        join_keys=["CUSTOMER_ID"],
        description="Customer entity for retail domain"
    )
    # Create feature view
    view = fs.add_feature_view(config, behavior_df, "CUSTOMER")
````



### Feature Transformations

Common transformations are built in:

```python
from snowflake_feature_store.transforms import fill_na, date_diff, window_agg

# Fill missing values
df = fill_na(['session_length'], 0)(df)

# Calculate temporal features
df = date_diff('signup_date', 'days_since_signup')(df)

# Window aggregations
df = window_agg(
    {'amount': ['SUM', 'AVG']},
    partition_by='customer_id',
    order_by='date',
    window_size=7
)(df)
```

### Getting Training Data

Generate training datasets with point-in-time correctness:


```python
training_data = fs.get_features(
    spine_df=customers_df,          # Base dataset
    feature_views=[behavior_config], # Features to include
    label_cols=['target'],          # Target variables
    spine_timestamp_col='date'      # For temporal correctness
)
```

## Core Concepts

### Feature Views

A feature view encapsulates related features and their transformations. For example:


```python
# Customer behavior features
behavior_config = FeatureViewConfig(
    name="customer_behavior",
    domain="RETAIL",
    entity="CUSTOMER",
    feature_type="BEHAVIOR",
    feature_descriptions={
        "avg_session_length": "Average session length in minutes",
        "total_purchases": "Total number of purchases"
    }
)
```

### Transforms
Transforms are composable functions that modify DataFrames. They're type-safe and can be chained:

```python
from snowflake_feature_store.transforms import Transform

def log_transform(col: str) -> Transform:
    "Take log of column values"
    return lambda df: df.with_column(
        f"LOG_{col}",
        F.log(F.col(col))
    )

# Use in pipeline
transforms = [
    fill_na(['amount'], 0),
    log_transform('amount')
]
```

## Documentation

The library is split into the following sections:

1. [Connection Management](./01_connection.ipynb): Setting up Snowflake connections
2. [Feature Transforms](./02_transforms.ipynb): Data transformation utilities
3. [Feature Views](./03_feature_view.ipynb): Feature configuration and management
4. [Feature Store](./04_manager.ipynb): High-level feature store operations
5. [Examples](./05_simple_examples.ipynb): End-to-end examples and patterns

## Advanced Usage

See the [examples notebook](./05_examples.ipynb) for:
- Window-based aggregations
- Custom transformations
- Point-in-time correct features
- Feature store callbacks
- Schema validation

## Contributing

PRs are welcome! Please see [CONTRIBUTING.md](CONTRIBUTING.md) for guidelines.




### Installation

Install latest from the GitHub [repository][repo]:

```sh
$ pip install git+https://github.com/Jeremy-Demlow/sf-feature-store.git
```

or from [pypi][pypi]


```sh
$ pip install sf_feature_store
```


[repo]: https://github.com/Jeremy-Demlow/sf-feature-store
[docs]: https://Jeremy-Demlow.github.io/sf-feature-store/
[pypi]: https://pypi.org/project/sf-feature-store/
[conda]: https://anaconda.org/Jeremy-Demlow/sf-feature-store